In [1]:
import duckdb
import pandas as pd

In [8]:
sql_query = '''
show tables
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())
/workspaces/data-engineering-with-data-build-tool-dbt-4458303/nyc_parking_violations
### gygughiu
###jbububn
##hvhjbkj

,summons_number,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,vehicle_expiration_date,violation_location,...,from_hours_in_effect,to_hours_in_effect,vehicle_color,unregistered_vehicle,vehicle_year,meter_number,feet_from_curb,no_standing_or_stopping_violation,hydrant_violation,double_parking_violation
0,9010912681,CA,PAS,2022-10-11,17,SUBN,FORD,T,20220788,50.0,...,0700A,0400P,BLACK,NaN,0,None,0,None,None,None
1,4858762841,NY,PAS,2023-08-21,36,4DSD,HONDA,V,0,NaN,...,None,None,GY,NaN,2003,None,0,None,None,None
2,4854645684,FL,PAS,2023-07-26,36,UT,BMW,V,0,NaN,...,None,None,WHI,NaN,2022,None,0,None,None,None
3,9044582707,NY,PAS,2023-04-10,21,SUBN,SUBAR,T,20231217,79.0,...,0900A,1030A,GY,NaN,2017,None,0,None,None,None
4,9041503330,NY,PAS,2023-03-21,21,4DSD,CHEVR,T,20250320,26.0,...,1100A,1230A,BK,NaN,2018,None,0,None,None,None


In [9]:
## When creating the dev datbase
sql_query_import_1='''
CREATE OR REPLACE TABLE parking_violation_codes AS
select *
FROM read_csv_auto(
'data/dof_parking_violation_codes.csv',
normalize_names=True)
'''
 
sql_query_import_2='''
CREATE OR REPLACE TABLE parking_violation_2023 AS
select *
FROM read_csv_auto(
'data/parking_violations_issued_fiscal_year_2023_sample.csv',
normalize_names=True
)
'''
 
with duckdb.connect('data/nyc_parking_violations.db') as con:
  con.sql(sql_query_import_1)
  con.sql(sql_query_import_2) 

In [2]:
## Creting proddatabase
sql_query_import_1='''
CREATE OR REPLACE TABLE parking_violation_codes AS
select *
FROM read_csv_auto(
'data/dof_parking_violation_codes.csv',
normalize_names=True)
'''

sql_query_import_2='''
CREATE OR REPLACE TABLE parking_violation_2023 AS
select *
FROM read_csv_auto(
'data/parking_violations_issued_fiscal_year_2023_sample.csv',
normalize_names=True
)
'''

with duckdb.connect('data/prod_nyc_parking_violations.db') as con:
  con.sql(sql_query_import_1)
  con.sql(sql_query_import_2) 

In [4]:
sql_query = '''
show tables
'''

with duckdb.connect('data/prod_nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())
### gygughiu
###jbububn
##hvhjbkj

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,first_model
3,gold_ticket_metrics
4,gold_vehicle_metrics
5,gold_vehicles_metrics
6,parking_violation_2023
7,parking_violation_codes
8,parking_violations_2023
9,ref_model
